In [6]:
import pandas as pd

gold=pd.read_csv("gold1.txt", sep='\t', names=["URL", "Category"])
gold

,URL,Category
0,http://0800-horoscope.com,G
1,http://18games.net,X
2,http://1pixelout.net,G
3,http://1porno.com/004/teens/index.html,X
4,http://1stpussy.com/005/16/index.html,X
...,...,...
1512,http://zipcodez.com,G
1513,http://zml.com,P
1514,http://zonezero.com,G
1515,http://zooped.com,R


In [7]:
labels=pd.read_csv("labels1.txt", sep='\t', names=["Turk", "URL", "Category"])
labels

,Turk,URL,Category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
...,...,...,...
92716,AFHMR93GDJCGT,http://zz868.com,G
92717,A13F119AMOA54H,http://zzx.cc,X
92718,A1S0C0EB7A6FFM,http://zzx.cc,P
92719,A21US576U8SCO4,http://zzx.cc,G


### Split thelabelsDataFrame into two:labelsongoldandlabelsunknown,the former containing all rows where the url is present in the gold set, andthe latter one contains all remaining rows oflabels.

In [17]:
labelsongold = labels.merge(gold, on=['URL'], suffixes=['_label','_gold'])
labelsongold

,Turk,URL,Category_label,Category_gold
0,A1253FXHCZ9CWM,http://0800-horoscope.com,G,G
1,A153PKAL7OAY36,http://0800-horoscope.com,G,G
2,A1FV9SAPL5C6KY,http://0800-horoscope.com,G,G
3,A1JTOT0DWM6QGL,http://0800-horoscope.com,G,G
4,A1PXXEOGQ76RNJ,http://0800-horoscope.com,G,G
...,...,...,...,...
3319,A35CBDDGBM7T6E,http://yousendit.com,G,G
3320,A3J86MK3VIE6ST,http://yousendit.com,G,G
3321,A4GP97LAQCCHJ,http://yousendit.com,G,G
3322,AAQNUEUU7H1Y5,http://yousendit.com,G,G


In [18]:
labelsunknown = labels.merge(labelsongold, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
labelsunknown

,Turk,URL,Category,Category_label,Category_gold,_merge
0,A1OT3A29R9N1DG,http://000.cc,P,NaN,NaN,left_only
1,A1PXXEOGQ76RNJ,http://000.cc,G,NaN,NaN,left_only
2,A1PXXEOGQ76RNJ,http://000.cc,G,NaN,NaN,left_only
3,A21US576U8SCO4,http://000.cc,G,NaN,NaN,left_only
4,A2LGX47NN7C5D3,http://000.cc,G,NaN,NaN,left_only
...,...,...,...,...,...,...
92730,AFHMR93GDJCGT,http://zz868.com,G,NaN,NaN,left_only
92731,A13F119AMOA54H,http://zzx.cc,X,NaN,NaN,left_only
92732,A1S0C0EB7A6FFM,http://zzx.cc,P,NaN,NaN,left_only
92733,A21US576U8SCO4,http://zzx.cc,G,NaN,NaN,left_only


Create aratergoodnessDataFrame that is indexed by turk, and has twocolumns:  the number of ratings, and the average correctness of ratings foreach turk (both on gold set urls).

In [21]:
labelsongold['Correctness']=(labelsongold['Category_gold']==labelsongold['Category_label'])

In [40]:
ratergoodness=labelsongold.groupby('Turk')['Correctness'].agg(['count','mean'])

In [44]:
ratergoodness.rename(columns={'count': 'the number of ratings','mean':'average correctness of ratings'},inplace=True)
ratergoodness

,the number of ratings,average correctness of ratings
Turk,,
A112DVP1KG4QZU,1,1.000000
A1253FXHCZ9CWM,29,0.517241
A12CY1Q7XKJJDE,1,1.000000
A12RE8G66WTO8B,20,0.750000
A12Y1GTGIQDGRA,3,0.333333
...,...,...
AYWOBMTEGFVVI,1,0.000000
AZ1S5BIG5R6P6,35,0.714286
AZSDW7JJL5VA7,100,0.680000


In [45]:
ratergoodness['odds']=ratergoodness['average correctness of ratings']/(1.001-ratergoodness['average correctness of ratings'])

In [46]:
ratergoodness

,the number of ratings,average correctness of ratings,odds
Turk,,,
A112DVP1KG4QZU,1,1.000000,1000.000000
A1253FXHCZ9CWM,29,0.517241,1.069214
A12CY1Q7XKJJDE,1,1.000000,1000.000000
A12RE8G66WTO8B,20,0.750000,2.988048
A12Y1GTGIQDGRA,3,0.333333,0.499251
...,...,...,...
AYWOBMTEGFVVI,1,0.000000,0.000000
AZ1S5BIG5R6P6,35,0.714286,2.491281
AZSDW7JJL5VA7,100,0.680000,2.118380


In [58]:
mask=ratergoodness['the number of ratings']>=20
ratergoodness[mask].sort_values(by=['odds'], ascending=False).head(10)

,the number of ratings,average correctness of ratings,odds
Turk,,,
A2U0R4X38GUKZE,20,0.950000,18.627451
A22C0PJUBFJTI0,36,0.916667,10.869565
A23YQUBXZPKILZ,24,0.875000,6.944444
ATVALOQVDCMZW,103,0.854369,5.826657
A1HIXWH4OXT8S4,40,0.825000,4.687500
A3220HG1O83HQ4,22,0.818182,4.475385
A32W20KGQXS0LL,25,0.800000,3.980100
A20PWAB7G3HDHU,20,0.800000,3.980100
AJSJVK40F5HM6,28,0.785714,3.649635


In [ ]:
#run regression